![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DeBertaForQuestionAnswering.ipynb)

## Import ONNX DeBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DeBertaForQuestionAnswering` is only available since in `Spark NLP 5.2.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBerta models trained/fine-tuned for question answering via `DeBertaForQuestionAnswering` or `TFDeBertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `DeBerta` in their labels
- Reference: [TFDeBertaForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/deberta#transformers.TFDebertaForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=deberta&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.51.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q transformers[onnx]==4.51.3 optimum onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models.
- We'll use [nbroad/deberta-v3-xsmall-squad2](https://huggingface.co/nbroad/deberta-v3-xsmall-squad2) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.
- In addition to the DeBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DebertaV2Tokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering

MODEL_NAME = 'nbroad/deberta-v3-xsmall-squad2'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/283M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/283M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


('onnx_models/nbroad/deberta-v3-xsmall-squad2/tokenizer_config.json',
 'onnx_models/nbroad/deberta-v3-xsmall-squad2/special_tokens_map.json',
 'onnx_models/nbroad/deberta-v3-xsmall-squad2/spm.model',
 'onnx_models/nbroad/deberta-v3-xsmall-squad2/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 279388
-rw-r--r-- 1 root root        23 Jun 11 22:39 added_tokens.json
-rw-r--r-- 1 root root       844 Jun 11 22:39 config.json
-rw-r--r-- 1 root root 283607260 Jun 11 22:39 model.onnx
-rw-r--r-- 1 root root       971 Jun 11 22:39 special_tokens_map.json
-rw-r--r-- 1 root root   2464616 Jun 11 22:39 spm.model
-rw-r--r-- 1 root root      1316 Jun 11 22:39 tokenizer_config.json


- We need to move `spm.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {ONNX_MODEL}/assets && mv {ONNX_MODEL}/spm.model {ONNX_MODEL}/assets

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/nbroad/deberta-v3-xsmall-squad2:
total 276984
-rw-r--r-- 1 root root        23 Jun 11 22:39 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 11 22:39 assets
-rw-r--r-- 1 root root       844 Jun 11 22:39 config.json
-rw-r--r-- 1 root root 283607260 Jun 11 22:39 model.onnx
-rw-r--r-- 1 root root       971 Jun 11 22:39 special_tokens_map.json
-rw-r--r-- 1 root root      1316 Jun 11 22:39 tokenizer_config.json

onnx_models/nbroad/deberta-v3-xsmall-squad2/assets:
total 2408
-rw-r--r-- 1 root root 2464616 Jun 11 22:39 spm.model


Voila! We have our `spm.model` inside assets directory

## Import and Save DeBertaForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 26.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `DeBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import DeBertaForQuestionAnswering

spanClassifier = DeBertaForQuestionAnswering.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DeBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 279416
-rw-r--r-- 1 root root 283650674 Jun 11 22:42 camembert_classification_onnx
-rw-r--r-- 1 root root   2464616 Jun 11 22:42 deberta_spp
drwxr-xr-x 2 root root      4096 Jun 11 22:42 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import DeBertaForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

span_classifier_loaded = DeBertaForQuestionAnswering.load(f"./{ONNX_MODEL}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline(stages=[
    document_assembler,
    span_classifier_loaded
])

example = spark.createDataFrame([
    ["What's my name?", "My name is Clara and I live in Berkeley."]
]).toDF("question", "context")

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, truncate=False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of `DeBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
